In [1]:
%matplotlib inline
import sys
sys.path.append('C:/YandexDisk/Work/RevoBigScorring/')

from tempfile import TemporaryFile
import pandas as pd
import numpy as np
import os
import getpass
import pyodbc
import datetime
import matplotlib
import matplotlib.pyplot as plt

from sklearn.utils import check_consistent_length, column_or_1d, check_array
import Buckets as bc
import woe
import scoringfunctions as sf
import bsfunctions as bs
import time
import random
from scipy import stats
from math import log

from sklearn.utils.fixes import isclose
from sklearn import metrics as met
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import KFold
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_selection import RFE
rcParams['figure.figsize'] = 7, 7
pd.options.mode.chained_assignment = None

In [268]:
import imp
sf = imp.reload(sf)

In [2]:
connectString = 'DRIVER={SQL Server};SERVER=dwh.int.revoplus.ru;DATABASE=Revo_DW;UID=e.migaev;PWD='
pswd = getpass.getpass('DWH_Password: ')
connectString = connectString + pswd
cnxn = pyodbc.connect(connectString)
pswd = connectString = ''

DWH_Password: ········


In [3]:
query = """select
	
	c.clientKey
	,c.guid
	,la.ApplicationDate
	,t.name as traderName
	,t.traderKey
    
    ,fa.[ApplicationId]
    ,fa.[CreditScore]
    ,fa.[CardId]
    ,fa.[FoStore]
    ,fa.[PosPolicyGroup]
    ,fa.[RegionStore]
    ,fa.[Age]
    ,fa.[CityStore]
    ,fa.[GenderTypeKey]
    ,fa.[InitialLimit]
    ,fa.[RetailerChain]
    ,fa.[RetailerPos]
    ,fa.[BkiFlg]
    ,fa.[PosBureauGroup]
    ,fa.[Segment]
    ,fa.[FinalDecision]
    ,fa.[PostBureau]
    ,fa.[PreBureau]
    ,fa.[FirstDeclineRule]
    ,fa.[Repeated]
    ,fa.[Limit]
    ,fa.[Source]
    ,fa.[Strategy]
    ,fa.[RiskGrade]
    ,fa.[ProbDef]
    ,fa.[ProbResp]
    ,fa.[MaxGapDays]
    ,fa.[MaxDifferenceInDays]
    ,fa.[TotalMaxPayment]
    ,fa.[TotalPaymentsCount]
    ,fa.[TotalPaymentsSum]
    ,fa.[LifePeriod]

	,fcr.[ApplicationId]
    ,fcr.[FirstLoanDate]
    ,fcr.[Inquiry12Month]
    ,fcr.[Inquiry1Month]
    ,fcr.[Inquiry1Week]
    ,fcr.[Inquiry3Month]
    ,fcr.[Inquiry6Month]
    ,fcr.[Inquiry9Month]
    ,fcr.[InquiryRecentPeriod]
    ,fcr.[LastLoanDate]
    ,fcr.[LoansActive]
    ,fcr.[LoansActiveMainBorrower]
    ,fcr.[LoansMainBorrower]
    ,fcr.[MaxOverdueStatus]
    ,fcr.[PayLoad]
    ,fcr.[TtlAccounts]
    ,fcr.[TtlBankruptcies]
    ,fcr.[TtlConsumer]
    ,fcr.[TtlCreditCard]
    ,fcr.[TtlDelq3059]
    ,fcr.[TtlDelq3059L12m]
    ,fcr.[TtlDelq30L12m]
    ,fcr.[TtlDelq5]
    ,fcr.[TtlDelq529]
    ,fcr.[TtlDelq6089]
    ,fcr.[TtlDelq6089L12m]
    ,fcr.[TtlDelq90Plus]
    ,fcr.[TtlDelq90PlusL12m]
    ,fcr.[TtlInquiries]
    ,fcr.[TtlLegals]
    ,fcr.[TtlOfficials]
    ,fcr.[WorstStatusEver]

from
	client c
		join
	(
		select
			*
			,Row_number() over (partition by clientKey order by ficoapplicationKey) as Farn
		from
			ficoapplication 
	)fa
on
	c.clientKey = fa.clientKey
		join
	loanApplication la
on
	la.loanApplicationKey = fa.loanApplicationKey
		join
	store s
on
	la.storeKey = s.storeKey
		join
	trader t
on
	t.traderKey = s.traderKey
		join
	(
		select
			*
			,Row_number() over (partition by ficoapplicationKey order by FicoCreditRegistryKey) as fcrRn
		from
			FicoCreditRegistry 
	)fcr
on
	fcr.ficoApplicationKey = fa.ficoApplicationKey
where
	faRn = 1 and fcrRn = 1 and la.ApplicationDate > '20160520'"""

In [4]:
df = pd.read_sql(query,cnxn)

In [24]:
df.to_csv('addScorringSrc.csv',index=False)


C:\Users\e.migaev\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
dfSrc = pd.read_csv('addScorringSrc.csv',encoding='cp1251')

C:\Users\e.migaev\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def MaxOverdueStatusClust(x):
    if x == 'A':
        return 0.0
    x = float(x)
    if x==0 :
        return 1.0
    elif x == 1:
        return -1.0
    else:
        return x
    
def WorstStatusEverClust(x):
    if x == 'A':
        return -1.0
    elif x== 'X':
        return 0.0
    x = float(x)
    if x==0.0 :
        return 1.0
    elif x==1.0:
        return -2.0
    else:
        return x
    
def mainTransform(df):
    dfVar = df[['FirstDeclineRule','ApplicationId','ApplicationDate','Age','GenderTypeKey','BkiFlg','FirstLoanDate','Inquiry12Month', 'Inquiry1Month',
       'Inquiry1Week', 'Inquiry3Month', 'Inquiry6Month', 'Inquiry9Month',
       'InquiryRecentPeriod', 'LastLoanDate', 'LoansActive',
       'LoansActiveMainBorrower', 'LoansMainBorrower', 'MaxOverdueStatus',
       'PayLoad', 'TtlAccounts', 'TtlConsumer',
       'TtlCreditCard', 'TtlDelq3059', 'TtlDelq3059L12m', 'TtlDelq30L12m',
       'TtlDelq5', 'TtlDelq529', 'TtlDelq6089', 
       'TtlDelq90Plus',  'TtlInquiries', 
        'WorstStatusEver','traderKey','clientKey']]

    dfVar = dfVar[dfVar.BkiFlg==1]
    dfVar = dfVar.drop(['BkiFlg'],axis=1)

    dfVar['ApplicationDate'] = pd.to_datetime( dfVar['ApplicationDate'])
    dfVar.FirstLoanDate = pd.to_datetime(dfVar.FirstLoanDate)
    dfVar.LastLoanDate = pd.to_datetime(dfVar.LastLoanDate)

    dfVar['floan'] = (dfVar.ApplicationDate - dfVar.FirstLoanDate).dt.days
    dfVar['lloan'] = (dfVar.ApplicationDate - dfVar.LastLoanDate).dt.days
    dfVar.loc[dfVar.floan<0,'floan'] = 0
    dfVar.loc[dfVar.lloan<0,'lloan'] = 0

    dfVar.MaxOverdueStatus = dfVar.MaxOverdueStatus.apply(MaxOverdueStatusClust)
    dfVar.WorstStatusEver = dfVar.WorstStatusEver.apply(WorstStatusEverClust)

    dfVar['TtlDelq30L12m'] = dfVar['TtlDelq30L12m'].fillna(0)
    dfVar['LoansMainBorrower'] = dfVar['LoansMainBorrower'].fillna(0)
    dfVar['TtlAccounts'] = dfVar['TtlAccounts'].fillna(0)
    dfVar['TtlDelq3059'] = dfVar['TtlDelq3059'].fillna(0)
    dfVar['TtlDelq3059'] = dfVar['TtlDelq3059'].fillna(0)

    dfVar[['TtlDelq30L12m','LoansMainBorrower','TtlAccounts','TtlDelq3059','TtlDelq3059','TtlDelq3059L12m','TtlDelq5',
          'TtlDelq529','TtlDelq6089','TtlDelq90Plus','TtlDelq3059L12m',
           'TtlInquiries','PayLoad','TtlDelq3059L12m']] = dfVar[['TtlDelq30L12m','LoansMainBorrower','TtlAccounts','TtlDelq3059','TtlDelq3059','TtlDelq3059L12m','TtlDelq5',
          'TtlDelq529','TtlDelq6089','TtlDelq90Plus','TtlDelq3059L12m',
           'TtlInquiries','PayLoad','TtlDelq3059L12m']].fillna(0)
    return dfVar

In [7]:
dfTr = mainTransform(dfSrc)

In [8]:
woeOut = pd.read_csv('woeOut.csv')

In [9]:
intercept = 559.50687475581992

In [10]:
dfTrClean = dfTr.loc[dfTr.floan.notnull(),:]
dfTrDirty = dfTr.loc[dfTr.floan.isnull(),:]

In [14]:
woeOut

,variable,minVal,maxVal,scorValue,varInit
0,Age,18,22,-31.526808,Age
1,Age,23,25,-16.221757,Age
2,Age,26,37,-2.527139,Age
3,Age,38,50,10.954572,Age
4,Age,51,78,25.007579,Age
5,GenderTypeKey,1,1,-18.048845,GenderTypeKey
6,GenderTypeKey,2,2,6.078572,GenderTypeKey
7,Inquiry1Week,NaN,NaN,0.824424,Inquiry1Week
8,Inquiry1Week,0,0,2.099308,Inquiry1Week
9,Inquiry1Week,1,19,-16.322608,Inquiry1Week


In [13]:
sf.getScoringTable(dfTrDirty,woeOut,intercept,'ApplicationId')

,ApplicationId,scoring,Age,GenderTypeKey,Inquiry1Week,Inquiry3Month,MaxOverdueStatus,TtlAccounts,TtlConsumer,TtlDelq30L12m,TtlDelq529,TtlDelq6089,WorstStatusEver,TtlDelq90Plus,floan
0,654518903,574.529589,-2.527139,6.078572,2.099308,3.096777,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
1,664192171,581.739969,10.954572,6.078572,0.824424,-1.899671,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
2,274628553,574.529589,-2.527139,6.078572,2.099308,3.096777,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
3,139491423,574.529589,-2.527139,6.078572,2.099308,3.096777,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
4,111404232,554.563639,-16.221757,6.078572,0.824424,-1.899671,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
5,200905229,568.258257,-2.527139,6.078572,0.824424,-1.899671,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
6,473329594,538.324611,-2.527139,-18.048845,2.099308,-8.980784,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
7,221775626,530.335494,-16.221757,6.078572,-16.322608,-8.980784,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
8,836701912,539.258589,-31.526808,6.078572,0.824424,-1.899671,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724
9,774760696,588.011301,10.954572,6.078572,2.099308,3.096777,0.31858,-17.169023,-9.987452,3.030001,3.965612,1.551005,33.550696,3.997501,-12.981724


In [81]:
dfTr.loc[dfTr.floan.isnull(),'ApplicationId'].values

array([654518903, 664192171, 274628553, 139491423, 111404232, 200905229,
       473329594, 221775626, 836701912, 774760696, 256742984, 339608544,
       458272763, 417636523], dtype=int64)

In [44]:
dfTrClean['Scoring'] = dfTrClean.apply(sf.getScoringColumn,args=[woeOut,intercept],axis=1)

In [47]:
mScore = pd.read_csv('mishaScore.csv',sep='\t')

In [55]:
myScore = dfTrClean[['ApplicationId','Scoring']].rename(columns = {'ApplicationId':'ApplicationID'})

In [58]:
compare = pd.merge(mScore,myScore,how='left')

In [63]:
compare['diff'] = (compare.score - compare.Scoring).apply(lambda x:abs(x))

In [68]:
comps = compare[compare.score>0].sort_values('diff',ascending=False)

In [73]:
ss = comps[(comps['diff']<60)&(comps['diff']>1)]

In [83]:
needCol = ss.ApplicationID.values

In [76]:
comps[(comps['diff']<60)]

,ApplicationID,score,Scoring,diff
3525,559717957,605.47,576.244340,29.225660
1392,255534429,631.63,655.749325,24.119325
4013,992237277,519.91,538.326478,18.416478
1835,43453574,589.57,577.420116,12.149884
1658,564369418,543.91,552.884460,8.974460
4302,207500539,504.78,504.755738,0.024262
4592,376579377,507.12,507.095998,0.024002
2948,428965865,530.48,530.456188,0.023812
600,237919097,496.27,496.246302,0.023698
2469,689606444,463.55,463.526402,0.023598


In [86]:
testDf = dfTr[dfTr.ApplicationId.isin(needCol)]

In [88]:
testDf['Scoring'] = testDf.apply(getScoringColumn,args=[woeOut,intercept],axis=1)

In [270]:
a = sf.getScoringTable(testDf,woeOut,intercept,'ApplicationId')
a.to_excel('scorDifference.xlsx',index=False)